## **Uso de LSTM para generar secuencias de texto: DON QUIJOTE DE LA MANCHA**

- Modelos de lenguaje, dada una serie de palabras, predecir (probabilistico) la proxima palabra o secuencia
- Sampling estocástico (elegir random en base a la probabilidad), cuyas ventajas son:
     - Produce frases más interesantes (creatividad)
     - Parámetro que controla esta estocasticidad es temperatura


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Cargamos texto del Quijote
with open("/kaggle/input/25agostocuentos/veinticinco_agosto_cuentos.txt",'r') as f:
    replace_chars = {'\n': ' ',
                    '\xad':'',
                    '±':'',
                    '³':'',
                    'º':'',
                    '¼':'',
                    'â':'a',
                    'ã':'a',
                    '©':''
    }
    corpus = f.read().lower()
    for old, new in replace_chars.items():
        corpus = corpus.replace(old,new)

# Vectorizar el texto

maxlen = 60 #longitud de las secuencias
step = 3 # cada cuantas letras empezar una secuencia

sentences = []
next_chars = []

for i in range(0,len(corpus) - maxlen,step):
    sentences.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])

In [4]:
# Vamos a crear nuestro corpus de letras
unique_chars = sorted(list(set(corpus)))
char_indices = {char : i for i,char in enumerate(unique_chars) }
print(char_indices)

{' ': 0, '!': 1, '(': 2, ')': 3, ',': 4, '-': 5, '.': 6, '/': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, ':': 18, ';': 19, '<': 20, '?': 21, '[': 22, ']': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49, '¡': 50, 'ª': 51, '«': 52, '»': 53, '¿': 54, 'á': 55, 'ä': 56, 'ç': 57, 'é': 58, 'ê': 59, 'í': 60, 'ñ': 61, 'ó': 62, 'ô': 63, 'ú': 64, 'ü': 65, '—': 66, '’': 67, '“': 68, '”': 69, '…': 70}


In [5]:
import numpy as np

# Vectorizacion (one hot encoding)
x = np.zeros((len(sentences), maxlen,len(unique_chars)), dtype=np.bool) # cada secuencia, hot encoded
y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool) # para cada secuencia, el siguiente caracter hot encoded
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char_indices[char]] = 1
    y[i,char_indices[next_chars[i]]] = 1
print(x.shape)
print(y.shape)

/tmp/ipykernel_32/1896659240.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen,len(unique_chars)), dtype=np.bool) # cada secuencia, hot encoded
/tmp/ipykernel_32/1896659240.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool) # para cada secuencia

(52873, 60, 71)
(52873, 71)


In [6]:
# Modelo con LSTM
from tensorflow.keras import layers
from tensorflow.keras import models

# Cuando se concatenan LSTM, parametro return_sequences=True excepto en la ultima
# Las capas intermedias pasan toda la secuencia de outputs, pero la ultima solo pasa el ultimo output
model = models.Sequential()
model.add(LSTM(256, input_shape=(maxlen,len(unique_chars)),return_sequences=True)) # devuelve una secuencia de vectores de 128 dimensiones
model.add(LSTM(64))
model.add(layers.Dense(len(unique_chars), activation='softmax')) # softmax para que el output sume 1

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 256)           335872    
                                                                 
 lstm_1 (LSTM)               (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 71)                4615      
                                                                 
Total params: 422,663
Trainable params: 422,663
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Entrenar el modelo
history = model.fit(x,y,batch_size=128,epochs=30)

Epoch 1/30
414/414 [==============================] - 12s 14ms/step - loss: 3.0295 - acc: 0.1752
Epoch 2/30
414/414 [==============================] - 6s 14ms/step - loss: 2.8853 - acc: 0.2163
Epoch 3/30
414/414 [==============================] - 6s 14ms/step - loss: 2.5778 - acc: 0.2635
Epoch 4/30
414/414 [==============================] - 6s 14ms/step - loss: 2.4266 - acc: 0.2888
Epoch 5/30
414/414 [==============================] - 6s 14ms/step - loss: 2.3339 - acc: 0.3149
Epoch 6/30
414/414 [==============================] - 6s 14ms/step - loss: 2.2617 - acc: 0.3326
Epoch 7/30
414/414 [==============================] - 6s 14ms/step - loss: 2.2066 - acc: 0.3489
Epoch 8/30
414/414 [==============================] - 6s 14ms/step - loss: 2.1607 - acc: 0.3584
Epoch 9/30
414/414 [==============================] - 6s 14ms/step - loss: 2.1214 - acc: 0.3673
Epoch 10/30
414/414 [==============================] - 6s 14ms/step - loss: 2.0842 - acc: 0.3772
Epoch 11/30
414/414 [=================

In [8]:
import numpy as np

def transform_distribution(predictions, temperature=0.5):
    predictions = np.array(predictions).astype('float64')
    # temperaturas altas: mas entropia (mas aleatorio)
    # temperaturas bajas: menos estocasticidad (mas deterministico)
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions) # asegurarse que los valores suman 1 (probabilidad)
    probs = np.random.multinomial(1, predictions, 1)
    return np.argmax(probs)

In [12]:
import random
# generar secuencias arbitrarias de texto
predict_length = 400
temperature = 0.5   #MIS NOTAS: bajarle este valor para que sea menos creativo
# random text seed
start_index = random.randint(0,len(corpus) - maxlen - 1)
input_text = corpus[start_index: start_index + maxlen]
print('Seed: ' + input_text)
generated_text = input_text

for i in range(predict_length):
    sampled = np.zeros((1, maxlen, len(unique_chars)))
    for t, char in enumerate(input_text):
        sampled[0,t,char_indices[char]] = 1.

    prediction = model.predict(sampled, verbose=0)[0]
    next_index = transform_distribution(prediction,temperature)
    next_char = unique_chars[next_index]

    #pegar el nuevo texto
    input_text += next_char
    generated_text += next_char
    input_text = input_text[1:]

generated_text

Seed: r un camino de la llanura. me pregunté sin mucha curiosidad 


'r un camino de la llanura. me pregunté sin mucha curiosidad de la pespericidad de la peesta.  —en una coma la insimena los deligaciones de la liertat de la labra de la presidia del conve de las palabras.  la con la inferedad de la convela y de la peleración del ispertirio de la persada mentada a se losor y la pendía es para del congrena o por la persado de la persa de la constanada de cada estrabaran de la premida por la convercidad de la peesta.  —no ente'